In [170]:
import pandas as pd
import numpy as np
import plotly.express as px

In [171]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [172]:
df.duplicated().sum()


0

In [173]:
df.describe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [174]:
df.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [175]:
px.imshow(df.corr(),text_auto='.02')

In [176]:
df.isnull().sum()


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [177]:

df[[ 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']] = df[[ 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']].replace(0, np.NaN)

In [178]:
df.isna().sum()


Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [179]:
def median_target(var):   
    temp = df[df[var].notnull()]
    
    median_values = temp.groupby('Outcome')[[var]].median().reset_index()
    
    return median_values

In [180]:
columns = df.columns.drop("Outcome","Pregnancies")

for i in columns:
    median_vals = median_target(i)
    
    df.loc[(df['Outcome'] == 0) & (df[i].isnull()), i] = median_vals.loc[median_vals['Outcome'] == 0, i].values[0]
    
    df.loc[(df['Outcome'] == 1) & (df[i].isnull()), i] = median_vals.loc[median_vals['Outcome'] == 1, i].values[0]

In [181]:
df.isna().sum()


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [182]:
outlier_feature=[]
for col in df:
    q1=df[col].quantile(0.25)
    q2=df[col].quantile(0.5)
    q3=df[col].quantile(0.75)
    iqr=q3-q1
    low=q1-iqr*1.5
    high=q3+1.5*iqr
    if df[(df[col]>high)].any(axis=None):
        print(col,": yes")
        outlier_feature.append(str(col))
    else:
        print(col," : no")

Pregnancies : yes
Glucose  : no
BloodPressure : yes
SkinThickness : yes
Insulin : yes
BMI : yes
DiabetesPedigreeFunction : yes
Age : yes
Outcome  : no


In [183]:
outlier_feature

['Pregnancies',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [184]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=10)
lof.fit_predict(df)

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
        1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1

In [185]:
df_scores = lof.negative_outlier_factor_
np.sort(df_scores)[0:20]

array([-3.30445978, -3.05893469, -2.48884101, -2.37289269, -2.15297995,
       -2.09708735, -2.0772561 , -2.01096252, -1.95255968, -1.86384019,
       -1.74003158, -1.72703492, -1.71674689, -1.70343883, -1.6688722 ,
       -1.64296768, -1.64190437, -1.61620872, -1.61369917, -1.60057603])

In [186]:
thresold = np.sort(df_scores)[7]
thresold

-2.0109625157063844

In [187]:
outlier = df_scores>thresold
df = df[outlier]
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [188]:
NewBMI=pd.Series(["UnderWeight","Normal","OverWeight","Obesity1","Obesity2","Obesity3"])
NewBMI


0    UnderWeight
1         Normal
2     OverWeight
3       Obesity1
4       Obesity2
5       Obesity3
dtype: object

In [189]:
df["NewBMI"]=NewBMI

df.loc[df["BMI"]<18.5,"NewBMI"]=NewBMI[0]
df.loc[(df["BMI"]>18.5) & (df["BMI"]< 24.9),"NewBMI"]=NewBMI[1]
df.loc[(df["BMI"]>24.9) & (df["BMI"]< 29.9),"NewBMI"]=NewBMI[2]
df.loc[(df["BMI"]>29.9) & df["BMI"]<=34.9, "NewBMI"] = NewBMI[3]
df.loc[(df["BMI"]>34.9) & df["BMI"]<=39.9, "NewBMI"] = NewBMI[4]
df.loc[df["BMI"]>39.9, "NewBMI"] = NewBMI[5]


In [190]:
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1,Obesity2
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0,Obesity2
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1,Obesity2
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0,Obesity2
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1,Obesity3


In [191]:
df["NewBMI"].value_counts()


Obesity2    665
Obesity3     95
Name: NewBMI, dtype: int64

In [192]:
def set_Insulin(row):
    if row>=16 and row<=166:
        return "Normal"
    else:
        return "AbNormal"

In [193]:
df["NewInsulin"]=df["Insulin"].apply(lambda x : set_Insulin(x))
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI,NewInsulin
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1,Obesity2,AbNormal
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0,Obesity2,Normal
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1,Obesity2,AbNormal
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0,Obesity2,Normal
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1,Obesity3,AbNormal


In [194]:
NewGlucose = pd.Series(["Low", "Normal", "Overweight", "Secret", "High"], dtype = "category")
df["NewGlucose"] = NewGlucose

In [195]:
df.loc[df["Glucose"] <= 70, "NewGlucose"] = NewGlucose[0]
df.loc[(df["Glucose"] > 70) & (df["Glucose"] <= 99), "NewGlucose"] = NewGlucose[1]
df.loc[(df["Glucose"] > 99) & (df["Glucose"] <= 126), "NewGlucose"] = NewGlucose[2]
df.loc[df["Glucose"] > 126 ,"NewGlucose"] = NewGlucose[3]
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI,NewInsulin,NewGlucose
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1,Obesity2,AbNormal,Secret
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0,Obesity2,Normal,Normal
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1,Obesity2,AbNormal,Secret
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0,Obesity2,Normal,Normal
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1,Obesity3,AbNormal,Secret


In [196]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd


OHE = OneHotEncoder()

new_cols = df[["NewBMI", "NewInsulin", "NewGlucose"]]

NEWcols = OHE.fit_transform(new_cols)

NEWcols_df = pd.DataFrame(NEWcols.toarray(), columns=OHE.get_feature_names_out(new_cols.columns))

NEWcols_df.head()

,NewBMI_Obesity2,NewBMI_Obesity3,NewInsulin_AbNormal,NewInsulin_Normal,NewGlucose_Low,NewGlucose_Normal,NewGlucose_Overweight,NewGlucose_Secret
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [197]:
df=pd.concat([df,NEWcols_df],axis=1)
df.dropna(inplace=True)

In [198]:
df.drop(['NewBMI','NewInsulin', 'NewGlucose'],axis=1,inplace=True)
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'NewBMI_Obesity2',
       'NewBMI_Obesity3', 'NewInsulin_AbNormal', 'NewInsulin_Normal',
       'NewGlucose_Low', 'NewGlucose_Normal', 'NewGlucose_Overweight',
       'NewGlucose_Secret'],
      dtype='object')

In [199]:
categorical_df = df[['NewBMI_Obesity2',
       'NewBMI_Obesity3', 'NewInsulin_AbNormal', 'NewInsulin_Normal',
       'NewGlucose_Low', 'NewGlucose_Normal', 'NewGlucose_Overweight',
       'NewGlucose_Secret']]

In [200]:
x=df.drop("Outcome",axis=1)
y=df["Outcome"]

In [201]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [202]:
from sklearn.preprocessing import scale, StandardScaler
scaler =StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [203]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(class_weight="balanced",criterion = 'entropy', max_depth = 15, max_features = 0.75, min_samples_leaf = 2, min_samples_split = 3, n_estimators = 130)


In [204]:
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score,f1_score, roc_auc_score, roc_curve, classification_report,recall_score

model.fit(x_train,y_train)
preds=model.predict(x_test)
acc=accuracy_score(preds,y_test)
print(acc,"\n")
rec=recall_score(preds,y_test)
print(rec,"\n")
f1=f1_score(preds,y_test)
print(f1,"\n")
fig=px.imshow(confusion_matrix(preds,y_test),text_auto='.02')
fig.show()


0.9006622516556292 

0.9069767441860465 

0.8387096774193548 

